# Fractal Dimensions

This is currently a scrap / exploratory notebook for some measures. 

Current messsy code below copies implementations:
- Higuchi Fractal Dimension
- Petrosian Fractal Dimension

In [16]:
import numpy as np

from neurodsp.sim import *

In [11]:
# General simulation settings
n_seconds = 10
fs = 1000

In [12]:
# Specific settings
exp = -2

In [13]:
sig = sim_powerlaw(n_seconds, fs, exp)

## HFD

In [18]:
def hfd(X, Kmax):
    """Compute Higuchi Fractal Dimension of a time series X.
    kmax is an HFD parameter.

    NOTE: copied from pyeeg
    https://github.com/forrestbao/pyeeg/blob/master/pyeeg/fractal_dimension.py#L4
    """

    L = []
    x = []
    N = len(X)

    for k in range(1, Kmax):

        Lk = []

        for m in range(0, k):
            Lmk = 0
            for i in range(1, int(np.floor((N - m) / k))):
                Lmk += abs(X[m + i * k] - X[m + i * k - k])
            Lmk = Lmk * (N - 1) / np.floor((N - m) / float(k)) / k
            Lk.append(Lmk)

        L.append(np.log(np.mean(Lk)))
        x.append([np.log(float(1) / k), 1])

    (p, _, _, _) = np.linalg.lstsq(x, L)

    return p[0]

In [24]:
hfd(sig, 20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


0.47230163936614866

## PFD

In [25]:
def pfd(X, D=None):
    """Compute Petrosian Fractal Dimension of a time series from either two cases below:
        1. X, the time series of type list (default)
        2. D, the first order differential sequence of X (if D is provided, recommended to speed up)

    In case 1, D is computed using Numpy's difference function.
    To speed up, it is recommended to compute D before calling this function
    because D may also be used by other functions whereas computing it here
    again will slow down.

    NOTE: copied from pyeeg
    https://github.com/forrestbao/pyeeg/blob/master/pyeeg/fractal_dimension.py#L26
    """

    if D is None:
        D = np.diff(X)
        D = D.tolist()

    N_delta = 0  # number of sign changes in derivative of the signal

    for i in range(1, len(D)):
        if D[i] * D[i - 1] < 0:
            N_delta += 1

    n = len(X)

    return np.log10(n) / (np.log10(n) + np.log10(n / n + 0.4 * N_delta))

## Another HFD

In [26]:
"""
Copied from:
https://github.com/inuritdino/HiguchiFractalDimension/blob/master/hfd.py

Higuchi Fractal Dimension according to:
T. Higuchi, Approach to an Irregular Time Series on the
Basis of the Fractal Theory, Physica D, 1988; 31: 277-283.
"""

import numpy as np

def hfd2(X, **kwargs):
    """
    Calculate Higuchi Fractal Dimension (HFD) for 1D data/series
    Input:
    X - input (time) series (must be 1D, to be converted into a NumPy array)
    Output:

    HFD
    """
    k, L = curve_length(X, **kwargs)
    return lin_fit_hfd(k, L);


def curve_length(X, num_k=50, k_max=None):
    """
    Calculate curve length <Lk> for Higuchi Fractal Dimension (HFD)

    Input:

    X - input (time) series (must be 1D, to be converted into a NumPy array)
    num_k - number of k values to generate.
    k_max - the maximum k (the k array is generated uniformly in log space
            from 2 to k_max)

    Output:
    k - interval "times", window sizes
    Lk - curve length
    """
    ### Make sure X is a NumPy array with the correct dimension
    X = np.array(X)
    if X.ndim != 1:
        raise ValueError("Input array must be 1D (time series).")
    N = X.size

    # Get interval "time"
    k_arr = interval_t(N,num_val=num_k,kmax=k_max)

    # The average length
    Lk = np.empty(k_arr.size,dtype=np.float)

    for i in range(k_arr.size): # over array of k's
        Lmk = 0.0
        for j in range(k_arr[i]): # over m's
            ## Construct X_k^m, i.e. X_(k_arr[i])^j, as X[j::k_arr[i]]
            ## Calculate L_m(k)
            Lmk += (
                np.sum(np.abs(np.diff( X[j::k_arr[i]] )))
                * (N - 1) / ( ( (N-j-1)//k_arr[i] ) * k_arr[i])
            ) / k_arr[i]

        ### Calculate the average Lmk
        Lk[i] = Lmk / k_arr[i]

    return (k_arr, Lk);


def lin_fit_hfd(k,L,log=True):
    """Calculate Higuchi Fractal Dimension (HFD) by fitting a line to already computed
    interval times k and curve lengths L

    Input:
    k - interval "times", window sizes
    L - curve length
    log (=True) - k and L values will be transformed to np.log2(k) and np.log2(L), respectively

    Output:
    HFD
    """
    if log:
        return (-np.polyfit(np.log2(k),np.log2(L),deg=1)[0]);
    else:
        return (-np.polyfit(k,L,deg=1)[0]);


def interval_t(size,num_val=50,kmax=None):
    """Generate sequence of interval times, k"""

    if kmax is None:
        k_stop = size//2
    else:
        k_stop = kmax
    if k_stop > size//2:## prohibit going larger than N/2
        k_stop = size//2
        print("Warning: k cannot be longer than N/2")

    k = np.logspace(start=np.log2(2),stop=np.log2(k_stop),base=2,num=num_val,dtype=np.int)
    return np.unique(k);

In [28]:
hfd2(sig)

1.389425763032955